In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Learning Libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score
import seaborn as sns
from collections import Counter, defaultdict

Load Dataset and look at it

In [74]:
# from sklearn.preprocessing import MinMaxScaler
data = pd.read_csv("readMAST.csv")
# # List of columns to normalize
# columns_to_normalize = [
#     'prostate_vol', 'lesion_count', 'adc_kurt', 'adc_10p', 't2w_min', 'hbv_skew', 'psa', 'pirads_max'
# ]

# # Initialize the MinMaxScaler
# scaler = MinMaxScaler()

# scaled_data = scaler.fit_transform(data)

# # Convert scaled data back to DataFrame
# data = pd.DataFrame(scaled_data, columns=data.columns, index=data.index)

In [75]:

#remove user id from data frame
df = data.drop(columns=['contribution_months','4kscore', 'adc_50p', 'adc_75p', 'adc_90p'])
df = df.fillna(0)
#split train test 
from sklearn.model_selection import train_test_split
X = df
y = df.outcome.values.astype('int32')
X.drop(['outcome'], axis=1, inplace=True)
column = 'adc_min'

# Step 3: Calculate the average of the column excluding negative values
# Use boolean indexing to filter out negative values
positive_values = df[df[column] > 0][column]
average_value = positive_values.mean()

# Step 4: Replace negative values with the calculated average
df.loc[df[column] <= 0, column] = average_value
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.70, test_size=0.30, random_state=0)
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')



In [76]:
from numpy import loadtxt
import keras
from keras import layers
from keras import Sequential
# from keras import Dense
import tensorflow as tf
# from keras import Adam


In [77]:
# Method 1: Check for any missing values in the DataFrame
print("\nMissing values in the DataFrame:")
print(df.isnull().sum())

# Method 2: Filter rows with missing values
print("\nRows with missing values:")
print(df[df.isnull().any(axis=1)])

# Method 3: Get the total count of missing values in the DataFrame
print("\nTotal number of missing values in the DataFrame:")
print(df.isnull().sum().sum())
print("\nPositions of missing values:")
missing_positions = df.isnull()
for column in missing_positions.columns:
    for index, is_missing in missing_positions[column].items():
        if is_missing:
            print(f"Missing value at Column: {column}, Row: {index}")



Missing values in the DataFrame:
prostate_vol        0
lesion_count        0
total_lesion_vol    0
adc_mean            0
adc_min             0
adc_max             0
adc_sd              0
adc_skew            0
adc_kurt            0
adc_10p             0
adc_25p             0
t2w_mean            0
t2w_min             0
t2w_max             0
t2w_sd              0
t2w_skew            0
t2w_kurt            0
t2w_10p             0
t2w_25p             0
t2w_50p             0
t2w_75p             0
t2w_90p             0
hbv_mean            0
hbv_min             0
hbv_max             0
hbv_sd              0
hbv_skew            0
hbv_kurt            0
hbv_10p             0
hbv_25p             0
hbv_50p             0
hbv_75p             0
hbv_90p             0
dce_mean            0
dce_min             0
dce_max             0
dce_sd              0
dce_skew            0
dce_kurt            0
dce_10p             0
dce_25p             0
dce_50p             0
dce_75p             0
dce_90p             

In [5]:

X_train_sample = X_train[:10].values  # Convert a small portion of X_train to NumPy array
y_train_sample = y_train[:10]  # Use a small portion of y_train

print(type(X_train_sample), X_train_sample.shape)  # Check type and shape
print(type(y_train_sample), y_train_sample.shape)


<class 'numpy.ndarray'> (10, 51)
<class 'numpy.ndarray'> (10,)


In [55]:
print(X_train.head())  # Display the first few rows of X_train
print(X_train.dtypes)  # Check the data types of each column in X_train
print(y_train[:10])    # Display the first 10 values of y_train


     prostate_vol  lesion_count  total_lesion_vol  adc_mean   adc_min  \
27       0.096379      0.307692          0.044387  0.292344  0.528042   
121      0.369902      0.307692          0.149734  0.733633  0.747748   
86       0.157591      0.153846          0.218012  0.647460  0.610252   
61       0.437767      0.384615          0.368289  0.482384  0.413295   
106      0.164831      0.076923          0.061750  0.792023  0.820993   

      adc_max    adc_sd  adc_skew  adc_kurt   adc_10p  ...   dce_50p  \
27   0.192807  0.223005  0.439943  0.145431  0.405320  ...  0.001875   
121  0.332633  0.105842  0.567077  0.062032  0.789991  ...  0.004794   
86   0.275872  0.067210  0.584267  0.057891  0.749172  ...  0.004229   
61   0.344785  0.199857  0.683252  0.065299  0.559904  ...  0.001846   
106  0.309383  0.048024  0.501513  0.092959  0.874887  ...  0.001516   

      dce_75p   dce_90p       psa  pirads_max       bmi       age  nccn  race  \
27   0.001468  0.001232  0.030534         0.8  

In [78]:
def roc(real_labels, predicted_probability):
    tpr = []
    fpr = []
    thresh=0.0


    #for i in range(0,100):
    #    sens_spec.append([])


    for i in range(100):
        nTruePos = 0
        nPos = 0
        nFalsePos = 0
        nNeg = 0
        for j, label in enumerate(real_labels):
            #print("real_labels[i] " + str(real_labels[j]))
            if (label == 1):   # if a cancer image
                nPos += 1
                if (predicted_probability[j] > thresh):     # if we predict cancer
                    nTruePos += 1
            else:                          # a non-cancer image
                nNeg += 1
                if (predicted_probability[j] > thresh):
                    nFalsePos += 1
#         print("thresh " + str(thresh) + " nTruePos " + str(nTruePos) + " nPos " + str(nPos) + " nFalsePos " + str(nFalsePos) + " nNeg " + str(nNeg))
        tpr.append( float(nTruePos) / float(nPos) )
        fpr.append( float(nFalsePos) / float(nNeg) )
        thresh+=0.01
    return tpr, fpr


In [32]:
def main(num):
    loss_results = []
    accuracy_results = []
    nn_auc_results = []
    threshold_results = []
    nn_fpr_results =[]
    nn_tpr_results = []
    for i in range(num):
        nn_model = tf.keras.Sequential([
        tf.keras.Input(shape=(54,)),
        tf.keras.layers.Dense(8, activation=tf.nn.relu, input_shape=(54,)),  # input shape required
        tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
        ])
       
       
        opt=keras.optimizers.Adam(learning_rate=0.001)

        nn_model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
        nn = nn_model.fit(X_train, y_train, epochs=1000, batch_size=10, verbose=0)
        loss, accuracy = nn_model.evaluate(X_test, y_test, verbose=0)
        loss_results.append(loss)
        accuracy_results.append(accuracy)
        predictions = nn_model.predict(X_test)
        neuralNetworkAUC = metrics.roc_auc_score(y_test, nn_model.predict(X_test))
        nn_auc_results.append(neuralNetworkAUC)
        fpr_NN, tpr_NN, threshold = metrics.roc_curve(y_test, nn_model.predict(X_test))
        nn_fpr_results.append(fpr_NN)
        nn_tpr_results.append(tpr_NN)
        threshold_results.append(threshold)
    return loss_results, accuracy_results, nn_auc_results, threshold_results, nn_fpr_results, nn_tpr_results

    #neural_network_AUC = metrics.roc_auc_score(y_test, nn.predict(X_test, batch_size=None, verbose=0)[:,1])
    #fpr_nn, tpr_nn, threshold = metrics.roc_curve(y_test, nn.predict(X_test, batch_size=None, verbose=0)[:,1])


In [ ]:
#call the function
loss_results, accuracy_results, nn_auc_results, threshold_results, nn_fpr_results, nn_tpr_results = main(3)

In [79]:
def experiment(num, X, y):
    logistic_regression_results = []
    l_tpr = []
    l_fpr = []
    random_forest_results = []
    rf_tpr = []
    rf_fpr = []
    for i in range(num):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)
        
        #Random Forest Classifier
        rf_model = RandomForestClassifier(n_estimators = 100, max_depth=5, random_state = 42)
        rf = rf_model.fit(X_train, y_train)
        predictions = rf.predict(X_test)
        #find ROC AUC curve
        randomForestAUC = metrics.roc_auc_score(y_test, rf.predict_proba(X_test)[:,1])
        fpr_RF, tpr_RF, threshold = metrics.roc_curve(y_test, rf.predict_proba(X_test)[:,1])
        random_forest_results.append(randomForestAUC)
        rf_tpr.append(tpr_RF)
        rf_fpr.append(fpr_RF)
        #Logistic regression
        l_model = LogisticRegression(solver='lbfgs')
        log = l_model.fit(X_train, y_train)
        predictions = log.predict(X_test)
        #find roc auc curve
        logisticRegressionAUC = metrics.roc_auc_score(y_test, log.predict_proba(X_test)[:,1])
        fpr_log, tpr_log, threshold = metrics.roc_curve(y_test, log.predict_proba(X_test)[:,1])
        logistic_regression_results.append(logisticRegressionAUC)
        l_tpr.append(tpr_log)
        l_fpr.append(fpr_log)
    
    #return random_forest_results, rf_tpr, rf_fpr
    return logistic_regression_results, l_tpr, l_fpr, random_forest_results, rf_tpr, rf_fpr

In [80]:
logistic_regression_results, l_tpr, l_fpr, rf_result, rf_tpr_1, rf_fpr_1 = experiment(2000, X, y)

/opt/anaconda3/envs/inferences/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/envs/inferences/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

In [81]:
print(max(rf_result))

0.823076923076923


In [82]:
testing_data = pd.read_csv("MAST_testset.csv")
real_outcome = testing_data.outcome.values.astype('int32')
dataframe = testing_data.drop(columns=['id', 'contribution_months', 'status', 'time_of_progression_months', 'adjusted_time_of_progression_months', 'outcome'])
dataframe = dataframe.fillna(0)

In [67]:
# # Initialize the MinMaxScaler
# scaler = MinMaxScaler()

# scaled_dataframe = scaler.fit_transform(dataframe)

# # Convert scaled data back to DataFrame
# dataframe = pd.DataFrame(scaled_dataframe, columns=dataframe.columns, index=dataframe.index)

In [83]:
dataframe.head()

,prostate_vol,lesion_count,total_lesion_vol,adc_mean,adc_min,adc_max,adc_sd,adc_skew,adc_kurt,adc_10p,...,dce_skew,dce_kurt,dce_10p,dce_25p,dce_50p,dce_75p,dce_90p,psa,4k_score,pirads_max
0,45.436125,2,0.103250,1093.433792,905.279541,1237.146973,87.028030,-0.291313,-0.842138,959.817224,...,-0.311288,-0.642382,0.505765,0.609667,0.737905,0.829638,0.909800,2.3,0.0,4.0
1,74.883375,9,1.106875,1354.286067,1037.673706,1927.905273,176.533642,1.017604,0.650210,1175.527612,...,0.906572,-0.125332,0.453322,0.680949,0.982839,1.180359,2.022611,4.4,20.0,2.0
2,24.986250,2,1.130250,811.119194,401.570312,1304.245728,183.424787,0.610807,-0.207734,599.024182,...,0.912548,1.870999,1.509600,1.738877,1.999238,2.188229,2.476597,5.9,0.0,4.0
3,86.315040,3,1.157520,998.002721,766.342041,1407.688110,110.215177,0.136681,-0.331705,848.642419,...,0.267430,-0.429945,0.468230,0.514293,0.572222,0.652306,0.715981,9.4,89.0,4.0
4,39.956000,3,0.378875,924.211244,594.618774,1106.505127,88.878198,-1.032460,1.528995,780.398779,...,0.326561,-0.761813,0.626613,0.696075,0.987007,1.133509,1.391550,6.3,35.0,0.0


In [84]:
df.head()

,prostate_vol,lesion_count,total_lesion_vol,adc_mean,adc_min,adc_max,adc_sd,adc_skew,adc_kurt,adc_10p,...,dce_50p,dce_75p,dce_90p,psa,pirads_max,bmi,age,nccn,race,ethnicity
0,48.993125,3,1.030000,1103.198043,631.599976,1858.439941,232.550714,0.497482,-0.264728,810.439978,...,1.397138,1.642994,1.905708,5.21,4,31.0,72,0,1,2
1,26.050920,2,0.892920,1036.473071,563.831177,1615.390747,171.229441,0.407404,-0.303378,842.470593,...,1.050223,1.111797,1.199560,1.61,0,25.9,56,0,1,1
2,92.462125,4,1.619625,1036.039573,863.860229,1221.504761,75.261871,0.151853,-0.567523,939.823853,...,2.218269,2.455834,2.682768,14.60,0,23.6,68,2,1,2
3,32.327750,4,1.129625,1175.841112,574.110606,1574.797974,154.126693,-3.259458,24.488103,1044.649414,...,1.779869,2.025097,2.228157,1.60,2,26.1,49,0,2,2
4,116.396000,7,0.972125,1158.384901,839.992248,1591.943848,132.490154,0.093294,-0.445859,990.146851,...,1.040919,1.143387,1.219857,8.28,2,28.5,58,1,2,2


In [85]:
import joblib
def prepare_inference_data(new_data, feature_columns):
    missing_columns = set(feature_columns) - set(new_data.columns)
    if missing_columns:
        raise ValueError(f"New data is missing columns: {missing_columns}")
    return new_data[feature_columns]


def run_inference_on_new_data(new_data, model_type='rf'):
    # Load the model
    if model_type == 'rf':
        model = joblib.load('best_rf_model.joblib')
        feature_columns = model.feature_names_in_  # Get feature names from the trained model
    elif model_type == 'log':
        model = joblib.load('best_log_model.joblib')
        feature_columns = model.feature_names_in_  # Get feature names from the trained model
    else:
        raise ValueError("Invalid model_type. Choose 'rf' for Random Forest or 'log' for Logistic Regression.")
    
    # Prepare data for inference
    new_data = prepare_inference_data(new_data, feature_columns)
    
    # Run predictions
    predictions = model.predict(new_data)
    probabilities = model.predict_proba(new_data)[:, 1]  # Probability for class 1
    
    return predictions, probabilities


In [86]:
predictions_rf, probabilities_rf = run_inference_on_new_data(dataframe, model_type='rf')
predictions_log, probabilities_log = run_inference_on_new_data(dataframe, model_type='log')

In [87]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def evaluate_predictions(predictions, true_outcomes):
    accuracy = accuracy_score(true_outcomes, predictions)
    conf_matrix = confusion_matrix(true_outcomes, predictions)
    class_report = classification_report(true_outcomes, predictions)
    
    return accuracy, conf_matrix, class_report

# Evaluate Random Forest predictions
accuracy_rf, conf_matrix_rf, class_report_rf = evaluate_predictions(predictions_rf, real_outcome)

# Evaluate Logistic Regression predictions
accuracy_log, conf_matrix_log, class_report_log = evaluate_predictions(predictions_log, real_outcome)


/opt/anaconda3/envs/inferences/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/inferences/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/inferences/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le

In [88]:
print(accuracy_rf, conf_matrix_rf, class_report_rf)

0.6666666666666666 [[9 0]
 [5 1]]               precision    recall  f1-score   support

           0       0.64      1.00      0.78         9
           1       1.00      0.17      0.29         6

    accuracy                           0.67        15
   macro avg       0.82      0.58      0.53        15
weighted avg       0.79      0.67      0.58        15



In [45]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# Example: Load your data
# df = pd.read_csv('your_data.csv')
# X = df.drop('target', axis=1)
# y = df['target']

# For demonstration, let's create some dummy data
# Assume you have 54 features in X and binary target variable y


# Initialize the model
model = RandomForestClassifier()

# Train the model
model.fit(X_train, y_train)

# Get feature importances
importances = model.feature_importances_

# Create a DataFrame to hold feature names and their importance
feature_importances = pd.DataFrame({'feature': X.columns, 'importance': importances})

# Sort the DataFrame by importance in descending order
feature_importances = feature_importances.sort_values(by='importance', ascending=False)

# Select the top 8 features
top_features = feature_importances.head(8)

print("Top 8 features:")
print(top_features)

Top 8 features:
         feature  importance
45    pirads_max    0.067004
0   prostate_vol    0.049360
6         adc_sd    0.046060
44           psa    0.044949
8       adc_kurt    0.039682
47           age    0.034883
48          nccn    0.033424
26      hbv_skew    0.030198


In [ ]:
print(nn.history.keys())
# summarize history for accuracy
plt.plot(nn.history['accuracy'])
#plt.plot(nn.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(nn.history['loss'])
#plt.plot(nn.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
max_non_linear_index = nn_auc_results.index(max(nn_auc_results))
plt.figure(figsize=(15,10))
plt.plot([0,1], [0,1], 'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0,1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.plot(nn_fpr_results[max_non_linear_index], nn_tpr_results[max_non_linear_index], label='Neural Network (AUC = %0.4f)' % max(nn_auc_results))
plt.legend(loc='lower right')
plt.plot()

In [ ]:
from scipy.interpolate import interp1d
from scipy.interpolate import CubicSpline
max_linear_index = logistic_regression_results.index(max(logistic_regression_results))
max_non_linear_index = rf_result.index(max(rf_result))
max_nn_index = nn_auc_results.index(max(nn_auc_results))
# Define a new set of points to interpolate
x_new = np.linspace(0, 1, 100)

# Create interpolation functions
fpr_interp = interp1d(rf_fpr_1[max_non_linear_index], rf_tpr_1[max_non_linear_index], kind='linear')

# Apply interpolation to get smooth values
tpr_smooth = fpr_interp(x_new)

# Define a new set of points to interpolate
x_new = np.linspace(0, 1, 100)

# Create interpolation functions
fpr_interp = interp1d(l_fpr[max_linear_index], l_tpr[max_linear_index], kind='linear')

# Apply interpolation to get smooth values
l_tpr_smooth = fpr_interp(x_new)
# Define a new set of points to interpolate
x_new = np.linspace(0, 1, 100)

# Create interpolation functions
fpr_interp = interp1d(nn_fpr_results[max_linear_index], nn_tpr_results[max_linear_index], kind='linear')

# Apply interpolation to get smooth values
nn_tpr_smooth = fpr_interp(x_new)

In [ ]:
#Roc Auc Logistic and Random Forest Smooth
# ROC plot for max AUC Logistic and Random Forest
max_linear_index = logistic_regression_results.index(max(logistic_regression_results))
max_non_linear_index = rf_result.index(max(rf_result))
max_nn_index = nn_auc_results.index(max(nn_auc_results))
plt.figure(figsize=(15,10))
plt.plot([0,1], [0,1], 'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0,1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Clinical Radiomics Fused Progression Prediction Model with 54 features')
plt.plot(x_new, l_tpr_smooth, label='Logistic Regression (AUC = %0.4f)' % max(logistic_regression_results))
plt.plot(x_new, tpr_smooth, label='Random Forest (AUC = %0.4f)' % max(rf_result))
plt.plot(x_new, nn_tpr_smooth, label='Neural Network (AUC = %0.4f)' % max(nn_auc_results))
plt.legend(loc='lower right')
plt.plot()

In [ ]:
print('average AUC for random forest is:', sum(rf_result)/len(rf_result))
print('Max AUC for random forest is:', max(rf_result))
print('average AUC for logistic regression is:', sum(logistic_regression_results)/len(logistic_regression_results))
print('Max AUC for logistic regression is:', max(logistic_regression_results))
print('average AUC for Neural Networks is:', sum(nn_auc_results)/len(nn_auc_results))
print('Max AUC for Neural Networks is:', max(nn_auc_results))